# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: QLoRA
* Model: bert-base-cased
* Evaluation approach: 
* Fine-tuning dataset: nyu-mll/glue mrpc 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [2]:
# Load the MRPC validation dataset
eval_dataset = load_dataset('nyu-mll/glue', 'mrpc', split="validation")

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
# Load the pretrained base model (BERT) and tokenizer
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Tokenize the dataset using a lambda function
tokenized_eval_datasets = eval_dataset.map(
    lambda examples: tokenizer(
        examples['sentence1'], 
        examples['sentence2'], 
        truncation=True, 
        padding='max_length'
    ), 
    batched=True
)


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [5]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    fp16=True,
)


In [6]:
# Initialize Trainer
base_trainer = Trainer(
    model=base_model,
    args=training_args,
    eval_dataset=tokenized_eval_datasets,
    tokenizer=tokenizer,
)

# Evaluate the model
eval_results = base_trainer.evaluate()

print(f"BERT Evaluation results:\n{eval_results}")


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


BERT Evaluation results:
{'eval_loss': 0.7258169054985046, 'eval_runtime': 5.2457, 'eval_samples_per_second': 77.777, 'eval_steps_per_second': 4.956}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
# Import necessary libraries
# from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType #, prepare_model_for_kbit_training

In [8]:
# Load the MRPC training dataset
train_dataset = load_dataset('nyu-mll/glue', 'mrpc', split="train")

In [9]:
#Define BnB Cofiguration for 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# Load the pre-trained model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
    num_labels=2
)
# model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
# List the name of all the linear modules of the base model, on which the LoRA configuration will be applied
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.output.dense
bert.encoder.layer.1.attention.self.query
bert.encoder.layer.1.attention.self.key
bert.encoder.layer.1.attention.self.value
bert.encoder.layer.1.attention.output.dense
bert.encoder.layer.1.intermediate.dense
bert.encoder.layer.1.output.dense
bert.encoder.layer.2.attention.self.query
bert.encoder.layer.2.attention.self.key
bert.encoder.layer.2.attention.self.value
bert.encoder.layer.2.attention.output.dense
bert.encoder.layer.2.intermediate.dense
bert.encoder.layer.2.output.dense
bert.encoder.layer.3.attention.self.query
bert.encoder.layer.3.attention.self.key
bert.encoder.layer.3.attention.self.value
bert.encoder.layer.3.attention.output.dense
bert.encoder.layer.3.intermediate.dense
bert.encoder.layer.3.output.dense
bert.encoder.layer.4.attention.s

In [11]:
# Define LoRA Config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.2,
    target_modules=["query", "key", "value", "dense"],
    bias="none"
)

# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()


trainable params: 1,342,468 || all params: 110,824,708 || trainable%: 1.2113435931633585


In [12]:
# Preprocess the data
tokenized_train_datasets = train_dataset.map(
    lambda examples: tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding="max_length",
        truncation=True,
        max_length=128
    ),
    batched=True,
    remove_columns=["sentence1", "sentence2", "idx"]
)

tokenized_train_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_lora",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    bf16=False,
    fp16=True,
    report_to=None,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50
)


In [14]:
# Create Trainer instance
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_eval_datasets,
    tokenizer=tokenizer
)


In [15]:
# Train the model
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.615300,0.617240
2,0.578200,0.594308
3,0.549000,0.623815
4,0.490300,0.602974
5,0.492800,0.612165


TrainOutput(global_step=2295, training_loss=0.5477208854326235, metrics={'train_runtime': 290.789, 'train_samples_per_second': 63.07, 'train_steps_per_second': 7.892, 'total_flos': 1225251348787200.0, 'train_loss': 0.5477208854326235, 'epoch': 5.0})

In [16]:
# Evaluate the peft trained model before saving it
trainer.evaluate()

{'eval_loss': 0.5943083167076111,
 'eval_runtime': 7.1102,
 'eval_samples_per_second': 57.383,
 'eval_steps_per_second': 3.657,
 'epoch': 5.0}

In [17]:
# Save the fine-tuned LoRA model weights
peft_model.save_pretrained("bert_lora_model")
tokenizer.save_pretrained("bert_lora_model")

('bert_lora_model/tokenizer_config.json',
 'bert_lora_model/special_tokens_map.json',
 'bert_lora_model/vocab.txt',
 'bert_lora_model/added_tokens.json',
 'bert_lora_model/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [18]:
import pandas as pd
from peft import AutoPeftModelForSequenceClassification

In [19]:
#Load the saved peft model
peft_trained_model = AutoPeftModelForSequenceClassification.from_pretrained("bert_lora_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Initialize Trainers
peft_trainer = Trainer(
    model=peft_trained_model,
    args=training_args,
    eval_dataset=tokenized_eval_datasets,
    tokenizer=tokenizer,
)

# Evaluate the model
peft_results = peft_trainer.evaluate()

In [21]:
print(f"PEFT Trained Model Evaluation results:\n{peft_results}")

PEFT Trained Model Evaluation results:
{'eval_loss': 0.5943083167076111, 'eval_runtime': 7.0956, 'eval_samples_per_second': 57.5, 'eval_steps_per_second': 3.664}


In [22]:
# Create a DataFrame to compare the results
comparison_df = pd.DataFrame(
    {
        "Metric": ["Eval Loss"],
        "Base Model": [eval_results["eval_loss"]],
        "PEFT Trained Model": [peft_results["eval_loss"]]
    }
)

# Display the DataFrame
comparison_df

,Metric,Base Model,PEFT Trained Model
0,Eval Loss,0.725817,0.594308
